In [ ]:
with open("../data/nietzsche.txt", "r", encoding="utf8") as f:
    text = f.read().lower()

print(len(text))

In [ ]:
text[:500]

In [ ]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sequences = []
next_chars = []

for i in tqdm(range(0, len(text) - maxlen, step)):
    sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sequences)

In [ ]:
for i in range(5): print(repr(sequences[i])," : ", next_chars[i])

In [ ]:
chars = sorted(list(set(text)))
print(chars)

In [ ]:
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

In [ ]:
indices_char = dict((chars.index(char), char) for char in chars)
print(indices_char)

In [ ]:
X = []
y = []
for i, seq in tqdm(enumerate(sequences)):
    x = [char_indices[char] for char in seq]
    X.append(x)
    y.append([char_indices[next_chars[i]]])

X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
import tensorflow as tf

emb_size = 8
num_chars = len(chars)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_chars, emb_size, input_length=maxlen),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_chars, activation='softmax')
]) 

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()

In [ ]:
# with tf.device("/CPU:0"):
model.fit(X, y, epochs=1, batch_size=32)

In [ ]:
import random

start_index = random.randint(0, len(text) - maxlen - 1)
base_text = text[start_index: start_index + maxlen]
base_text

In [ ]:
import sys

next_char = 100
new_text = base_text
sys.stdout.write(new_text)

for _ in range(next_char):
    x = np.array([[char_indices[char] for char in base_text]])  
    predicted = np.argmax(model.predict(np.array([[char_indices[char] for char in new_text]]), verbose=0), axis=-1)
    new_char = indices_char[predicted[0]]
    new_text = new_text[1:] + new_char
    sys.stdout.write(new_char)